# 准备工作

In [1]:
'''
阿里研究院
阿里健康
阿里巴巴商学院
阿里数据

腾讯金融科技
腾讯研究院
腾讯媒体研究院
腾讯云启研究院
酷鹅用户研究院
'''
公众号 = "传媒志"

In [2]:
fn = { "output" : { "公众号_htm_snippets": "data_raw_src/公众号_htm_snippets_{公众号}.tsv",
                    "公众号_df": "data_raw_src/公众号_df_{公众号}.tsv",
                    "公众号_xlsx": "data_sets/公众号_url_{公众号}.xlsx" } \
      }

# 采集公众号（requests）

In [3]:
# 目标url

import time
import requests
import pandas as pd
import csv


url = "https://mp.weixin.qq.com/cgi-bin/appmsg"

# 使用Cookie，跳过登陆操作
headers = {
  
    "Cookie": "pgv_pvid=3950839744; ua_id=sYxPS3o9fUYxwbcdAAAAAMclyg6gESrtsvzX8Qzj6Po=; mm_lang=zh_CN; pgv_pvi=9712533504; tvfe_boss_uuid=fd0d0de8d635f508; RK=XXhx3TB8xN; ptcz=b7461990899925bba90a10635eab73277abf2ca17b44bff867c0804cbb1a6394; o_cookie=1021996837; pac_uid=1_1021996837; _ga=amp-ERRzzpI_KP93vslIHixerw; ied_qq=o1021996837; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%2216f32b521ee15f-0914669e0092ad-6701b35-2073600-16f32b521ef44f%22%2C%22%24device_id%22%3A%2216f32b521ee15f-0914669e0092ad-6701b35-2073600-16f32b521ef44f%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E5%BC%95%E8%8D%90%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22http%3A%2F%2F106.52.25.45%2F%22%2C%22%24latest_referrer_host%22%3A%22106.52.25.45%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC%22%7D%7D; XWINDEXGREY=0; ptui_loginuin=1021996837@qq.com; luin=o1021996837; openid2ticket_oQyw15_yHSNyGnMM1DsHQ6900QRQ=i8/OkgU2vQubPSB5+cq4f0FCYeAsjilQtpaHCXREqJA=; lskey=00010000648af4afcbe908249b5c92f7578d4b3d35a17f8e715ea8270548fd3a25ad5a7169f3863e8fa76f0a; pgv_si=s1051141120; uuid=691cb855def6055edc62ffb31b661149; rand_info=CAESIASAr6kU2gSomK9883Vsyf/wIYhs0zpp8CAPx29e30w5; slave_bizuin=3879021635; data_bizuin=3879021635; bizuin=3879021635; data_ticket=AIEcgAnNeHyRJQeqRbSOxEX7sqFj/qh65kGteCUMfJSVdZFyx9zM9Ki0N+ipzPar; slave_sid=V2xpel9yVjFta216UDBYY2dxSTI3QjNaYWNZd3NPU2paeDRNVllOcm10dG5ZZVRub1RnbmRReDNFejVaX0xUS1Y2TWJkeHBTOW9JT3JtcXdwempDRXlYMFNxYU9wWTlBNHNkTlE2YUtJNHkwRUo4T201TzVjS0V1RlBRa3cyNDF4dlJLY2s4TFZVUGlscHdJ; slave_user=gh_15a5ee2f3ff0; xid=5f68dad835b30325010d026440b5489a",
   
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"
}
data = {
    "token": "68703568",
    "lang": "zh_CN",
    "f": "json",
    "ajax": "1",
    "action": "list_ex",
    "begin": "0",
    "count": "5",
    "query": "",
    "fakeid": "MjM5OTE0ODA2MQ==",
    "type": "9",
}



content_list=[]

for i in range(7):
    data["begin"] = i*7
    print(data["begin"])
    time.sleep(3)
    # 使用get方法进行提交
    content_json = requests.get(url, headers=headers, params=data).json()
#     print(content_json)
    # 返回了一个json，里面是每一页的数据
    for item in content_json["app_msg_list"]:
    # 提取每页文章的标题及对应的url
        items = []
        items.append(item["title"])
        items.append(item["link"])
        items.append(item["create_time"])
        content_list.append(items)


name=['title','link','create_time']
test=pd.DataFrame(columns=name,data=content_list)
with pd.ExcelWriter(fn["output"]["公众号_xlsx"].format(公众号="腾讯研究院_requests_01")) as writer:
    test.to_excel(writer)

# test.to_csv("../微信公众号爬虫_zhichao/南方周末.csv",mode='a',encoding='utf-8')
# print("保存成功")

0


ProxyError: HTTPSConnectionPool(host='mp.weixin.qq.com', port=443): Max retries exceeded with url: /cgi-bin/appmsg?token=68703568&lang=zh_CN&f=json&ajax=1&action=list_ex&begin=0&count=5&query=&fakeid=MjM5OTE0ODA2MQ%3D%3D&type=9 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000020B420EC470>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。')))

# 采集公众号（selenium）

In [3]:
import pandas as pd
import numpy as np
from lxml.html import fromstring
import time
from random import random

# when selenium main_content is used
# Parses an HTML document from a string constant.  Returns the root nood
# root = fromstring(df.loc[1,"html_snippets"]) 

## 使用Selenium
* 要更改 opts.binary_location 至自己本地的Chrome浏览器，建议portable
* Chrome浏览器 和 chromedriver.exe要同版本号到小数后一位
* 要确保可以 开启浏览器机器人
* 要确保浏览器机器人 可以打开网页 driver.get("https://mp.weixin.qq.com")

In [4]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

#caps=dict()
#caps["pageLoadStrategy"] = "none"   # Do not wait for full page load

opts = webdriver.ChromeOptions()
opts.add_argument('--no-sandbox')#解决DevToolsActivePort文件不存在的报错
opts.add_argument('window-size=1920x3000') #指定浏览器分辨率
opts.add_argument('--disable-gpu') #谷歌文档提到需要加上一这个属性来规避bug
opts.add_argument('--hide-scrollbars') #隐藏滚动条, 应对些特殊页面
#opts.add_argument('blink-settings=imagesEnabled=false') #不加载图片, 提升速度
#opts.add_argument('--headless') #浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败

opts.binary_location = r"C:\Users\win10\AppData\Local\Google\Chrome\Application\chrome.exe" #"H:\_coding_\Gitee\InternetNewMedia\CapstonePrj2016\chromedriver.exe"  

# "H:\_coding_\Gitee\InternetNewMedia\CapstonePrj2016\chromedriver.exe"  
driver = webdriver.Chrome( chrome_options = opts) #desired_capabilities=caps,

C:\Users\win10\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


In [5]:
driver.get("https://mp.weixin.qq.com")

## 填表登入

selenium 的定位方法
* find_element_by_id &ensp;&ensp;&ensp;  根据标签id定位
* find_element_by_name   &ensp;&ensp;&ensp; 根据标签的name定位
* find_element_by_xpath  &ensp;&ensp;&ensp; 根据xpath定位
* find_element_by_link_text  &ensp;&ensp;&ensp; 通过文字链接来定位元素
* find_element_by_partial_link_text  &ensp;&ensp;&ensp;  通过文字链接来定位元素
* find_element_by_tag_name  &ensp;&ensp;&ensp;  根据标签的名字定位
* find_element_by_class_name  &ensp;&ensp;&ensp; 通过class name 定位
* find_element_by_css_selector  &ensp;&ensp;&ensp;  根据元素属性来定位

In [6]:
payload =  {"account": "15219720545@163.com", "password": "1021996837PQ"}
# payload =  {"account": "NFUHacks@163.com", "password": "NFU706947580"}
driver.find_element_by_xpath('//div[@class="login__type__container login__type__container__scan"]/a').click()

WebDriver 常用方法：
* clear()清楚文本
* send_keys(values)模拟按键输入
* click()模拟点击
* submit模拟提交

In [7]:
driver.find_element_by_xpath('//form[@class="login_form"]//input[@name="account"]').clear()
driver.find_element_by_xpath('//form[@class="login_form"]//input[@name="account"]').send_keys(payload['account'])
driver.find_element_by_xpath('//form[@class="login_form"]//input[@name="password"]').clear()
driver.find_element_by_xpath('//form[@class="login_form"]//input[@name="password"]').send_keys(payload['password'])

In [8]:
driver.find_element_by_xpath('//div[@class="login_btn_panel"]/a').click()

## 点选单

其他常用方法
* size：返回元素的尺寸
* text：获取元素的文本
* get_attribute：获取属性值  &ensp;&ensp;&ensp; get_attribute('innerHTML')获取元素内的全部HTML
* is_displayed()：设置该元素用户是否可见

In [9]:
element = driver.find_element_by_xpath('//a[@id="m_open"]')
element.click()
main_content = element.get_attribute('innerHTML')
main_content

'展开'

In [10]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

In [11]:
element = driver.find_element_by_xpath('//li[@title[contains(.,"素材管理")]]/a') 
# main_content = element.get_attribute('innerHTML')
# main_content
url_素材管理= element.get_attribute("href")
url_素材管理

'https://mp.weixin.qq.com/cgi-bin/appmsg?begin=0&count=10&t=media/appmsg_list&type=10&action=list&token=210474303&lang=zh_CN'

In [12]:
driver.get(url_素材管理)

## 新建图文消息

In [13]:
element = driver.find_element_by_xpath('//*[text()[contains(.,"新建图文消息")]]') 
main_content = element.get_attribute('innerHTML')
main_content
element.click()

In [14]:
print (driver.window_handles)

['CDwindow-0B86F03C31BFB753C69D826DCF952AB9', 'CDwindow-959F9A76508EE540D684FC1F20496A7B']


In [15]:
# 新建图文消息开了另一分视窗，所以要切换 switch_to 
driver.switch_to.window(driver.window_handles[-1])

## 超链接

In [16]:
element = driver.find_element_by_xpath('//*[text()[contains(.,"超链接")]]') 
main_content = element.get_attribute('innerHTML')
print(main_content)
element.click()

                超链接              


In [17]:
# 点 选择其他公众号
element = driver.find_element_by_xpath('//*[text()[contains(.,"选择其他公众号")]]') 
main_content = element.get_attribute('innerHTML')
print(main_content)
element.click()

选择其他公众号


In [18]:
driver.find_element_by_xpath('//form//div[@class="inner_link_account_area"]//input[@class="weui-desktop-form__input"]').clear()
driver.find_element_by_xpath('//form//div[@class="inner_link_account_area"]//input[@class="weui-desktop-form__input"]').send_keys(公众号)

In [19]:
# 点放大镜搜
element = driver.find_element_by_xpath('//button[@class="weui-desktop-icon-btn weui-desktop-search__btn"]')
main_content = element.get_attribute('innerHTML')
print(main_content)
element.click()

<div class="weui-desktop-icon weui-desktop-icon__inputSearch weui-desktop-icon__small"><!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <!----> <svg width="16" height="16" viewBox="0 0 16 16" xmlns="http://www.w3.org/2000/svg"><path d="M11.33 10.007l4.273 4.273a.502.502 0 0 1 .005.709l-.585.584a.499.499 0 0 1-.709-.004L10.046 11.3a6.278 6.278 0 1 1 1.284-1.294zm.012-3.729a5.063 5.063 0 1 0-10.127 0 5.063 5.063 0 0 0 10.127 0z"></path></svg> <!----> <!----> <!----> <!----></div>


In [20]:
element = driver.find_element_by_xpath('//ul[@class="inner_link_account_list"]')
main_content = element.get_attribute('innerHTML')
print(main_content)
公众号SERP = main_content


<li class="inner_link_account_item"><div class="weui-desktop-vm_primary"><img src="http://mmbiz.qpic.cn/mmbiz_png/Ns3uHG4PrNiaicLseGUeYpoyjJD4Oyh8InWQOsGeQ1bwW5h4iaXsyFFKAleZ8PKbabF5FnqrkWW0ZxT0uPHqya2lA/0?wx_fmt=png" class="inner_link_account_avatar"> <strong class="inner_link_account_nickname">传媒志</strong> <i class="inner_link_account_wechat">微信号：chuanmei12321</i></div> <div class="weui-desktop-vm_default inner_link_account_type">订阅号</div></li><li class="inner_link_account_item"><div class="weui-desktop-vm_primary"><img src="http://mmbiz.qpic.cn/mmbiz_png/8uvMyVhQoZUXRwnydegyQl5f97ciaPYRlls7nuK64tqxbroQiaIPZhhvraOAglAIlBGqZunmSvGNKZBlAmGz7znA/0?wx_fmt=png" class="inner_link_account_avatar"> <strong class="inner_link_account_nickname">传媒青年志</strong> <i class="inner_link_account_wechat">微信号：wcqnz2016</i></div> <div class="weui-desktop-vm_default inner_link_account_type">订阅号</div></li><li class="inner_link_account_item"><div class="weui-desktop-vm_primary"><img src="http://mmbiz.qpic.cn

In [21]:
# 解析
root = fromstring(公众号SERP) 

In [22]:
主 = root.xpath('//li[@class="inner_link_account_item"]')

account_list = []
for e in 主:
    account_nickname = e.xpath('./div/strong[@class="inner_link_account_nickname"]')[0].text
    account_wechat = e.xpath('./div/i[@class="inner_link_account_wechat"]')[0].text
    account_img = e.xpath('./div/img/@src')[0]
    account = {"nickname": account_nickname, "wechat": account_wechat, "img": account_img,}
    account_list.append(account)

df_account = pd.DataFrame(account_list)

In [23]:
df_account

,nickname,wechat,img
0,传媒志,微信号：chuanmei12321,http://mmbiz.qpic.cn/mmbiz_png/Ns3uHG4PrNiaicL...
1,传媒青年志,微信号：wcqnz2016,http://mmbiz.qpic.cn/mmbiz_png/8uvMyVhQoZUXRwn...
2,国际传媒志,微信号：未设置,http://mmbiz.qpic.cn/mmbiz_png/zVLJHq6uF01Y6Um...
3,长新传媒志,微信号：cxcmz2016,http://mmbiz.qpic.cn/mmbiz_png/HqDcgPYS8WDQiai...
4,高铁传媒志,微信号：gaotie-media,http://mmbiz.qpic.cn/mmbiz_png/yZZNiaNDNnud2bP...


In [24]:
element = driver.find_element_by_xpath('//ul[@class="inner_link_account_list"]/li')
main_content = element.get_attribute('innerHTML')
print(main_content)
element.click()

<div class="weui-desktop-vm_primary"><img src="http://mmbiz.qpic.cn/mmbiz_png/Ns3uHG4PrNiaicLseGUeYpoyjJD4Oyh8InWQOsGeQ1bwW5h4iaXsyFFKAleZ8PKbabF5FnqrkWW0ZxT0uPHqya2lA/0?wx_fmt=png" class="inner_link_account_avatar"> <strong class="inner_link_account_nickname">传媒志</strong> <i class="inner_link_account_wechat">微信号：chuanmei12321</i></div> <div class="weui-desktop-vm_default inner_link_account_type">订阅号</div>


In [25]:
# 跳转testing
'''
跳转_input = driver.find_element_by_xpath('//span[@class="weui-desktop-pagination__form"]/input')
跳转_a = driver.find_element_by_xpath('//span[@class="weui-desktop-pagination__form"]/a')
跳转_input.clear()
跳转_input.send_keys(2)
跳转_a.click()
'''

'\n跳转_input = driver.find_element_by_xpath(\'//span[@class="weui-desktop-pagination__form"]/input\')\n跳转_a = driver.find_element_by_xpath(\'//span[@class="weui-desktop-pagination__form"]/a\')\n跳转_input.clear()\n跳转_input.send_keys(2)\n跳转_a.click()\n'

In [26]:
# 跳转上限
l_e = driver.find_elements_by_xpath('//label[@class="weui-desktop-pagination__num"]')
l_e_int  = [int(x.text) for x in l_e] 
print (l_e_int)
print (l_e_int[0]==l_e_int[-1])

[1, 157]
False


In [27]:
pages = list(range(l_e_int[0],l_e_int[-1]+1 ))
#print(pages[0:2])
pages = list(range(1,l_e_int[-1]+1 ))
print(pages)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157]


## 循环/遍历

In [28]:
# global varialbes 
html_raw = dict()
main_content =""
element = None

In [29]:
def process_pages (pages):
    for p in pages:
        print (p,end='\t')

        跳转_input = driver.find_element_by_xpath('//span[@class="weui-desktop-pagination__form"]/input')
        跳转_a = driver.find_element_by_xpath('//span[@class="weui-desktop-pagination__form"]/a')
        跳转_input.clear()
        跳转_input.send_keys(p)
        跳转_a.click()

        time.sleep(45+120*random())

        element = driver.find_element_by_xpath('//div[@class="inner_link_article_list"]')
        main_content = element.get_attribute('innerHTML')
        #print(main_content)
        html_raw[p] = main_content

In [30]:
process_pages(pages)

1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	51	52	53	54	55	56	57	58	59	60	61	62	63	64	65	66	67	68	69	70	71	72	73	74	75	76	77	78	79	80	81	82	83	84	85	86	87	88	89	90	91	92	93	94	95	96	97	98	99	100	101	102	103	104	105	106	107	108	109	110	111	112	113	114	115	116	117	118	119	120	121	122	123	124	125	126	127	128	129	130	131	132	133	134	135	136	137	138	139	140	141	142	143	144	145	146	147	148	149	150	151	152	153	154	155	156	157	

In [31]:
df = pd.DataFrame([html_raw]).T
df.columns = ["html_snippets"]
df

,html_snippets
1,"<div><label class=""inner_link_article_item""><s..."
2,"<div><label class=""inner_link_article_item""><s..."
3,"<div><label class=""inner_link_article_item""><s..."
4,"<div><label class=""inner_link_article_item""><s..."
5,"<div><label class=""inner_link_article_item""><s..."
...,...
153,"<div><label class=""inner_link_article_item""><s..."
154,"<div><label class=""inner_link_article_item""><s..."
155,"<div><label class=""inner_link_article_item""><s..."
156,"<div><label class=""inner_link_article_item""><s..."


In [32]:
%store html_raw
import pickle 
filehandler = open("html_raw", 'wb') 
pickle.dump(html_raw, filehandler)

Stored 'html_raw' (dict)


In [33]:
df_out = df[~df.duplicated()]
print (len(df_out))
df[df.duplicated()]

156


,html_snippets
12,"<div><label class=""inner_link_article_item""><s..."


In [34]:
try_again = list(df[df.duplicated()].index)
print(try_again)
try_again = try_again + list (set(pages).difference(set(df.index.values)))
try_again

[12]


[12]

## 暂存档

In [35]:
filename = fn ["output"] ["公众号_htm_snippets"] 
df_out.to_csv(filename.format(公众号=公众号), sep="\t", encoding="utf8")

In [102]:
def parse_html_snippets(_snippet_):
    root = fromstring(_snippet_) 
    title = [x.text for x in root.xpath('//div[@class="inner_link_article_title"]')]
    create_time = [x.text for x in root.xpath('//div[@class="inner_link_article_date"]')]
    link = [x for x in root.xpath('//a/@href')]
    _df_ = pd.DataFrame({"标题":title, "发布时间": create_time, "链接":link})
    return(_df_)
    
l_df = []
for p in pages:
    _df_ = parse_html_snippets(df.loc[p,"html_snippets"])
    print (len(_df_), end=",")
    l_df.append(_df_)

5,7,6,7,10,5,7,6,5,7,7,7,6,7,9,7,7,6,10,8,7,7,9,6,8,8,6,6,9,9,9,8,9,7,9,8,9,7,7,9,7,6,6,8,7,7,6,7,7,8,7,9,8,8,8,6,7,7,8,7,9,7,8,10,9,10,7,7,9,8,9,9,10,8,9,9,10,10,8,8,8,9,10,9,10,10,8,10,8,9,10,9,7,9,10,9,9,10,10,10,10,13,10,9,9,10,10,10,10,9,9,10,9,10,10,9,8,8,10,9,10,9,7,9,8,10,9,10,9,8,10,9,9,10,9,10,8,9,6,7,8,10,9,9,9,10,6,5,5,5,5,5,5,5,5,5,3,

In [124]:
df_1000条 = pd.concat(l_df).reset_index(drop=True)
前1000章文章 = df_1000条.loc[0:1000]
前1000章文章

,标题,发布时间,链接
0,助你少走弯路！2020Fall美国传播与媒介研究类PhD申请分享,2020-04-10,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
1,“云监工”变“云守望” 守护和见证一座城市的复苏,2020-02-16,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
2,如果要用八个字总结2019年的互联网，你的答案是什么？,2020-01-07,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
3,“央视频”不一young：跨年“遥控器” 打造全国全民大联欢,2020-01-04,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
4,回顾与前瞻：新中国成立70年来的新闻传播史研究,2019-10-28,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
...,...,...,...
996,考研福利：2016年度热点事件学理性解读（下）,2016-12-23,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
997,再见，我去考研了！,2016-12-23,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
998,考研福利：2016年度热点事件学理性解读（上）,2016-12-22,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
999,雾霾成了中国大众文化的一部分,2016-12-22,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...


In [125]:
df_1000条.loc[0:1000].to_excel("1000章_传媒志_结果.xlsx",\
                sheet_name="传媒志结果")

In [59]:
df_前50页.tail(5)

,title,create_time,link
1270,一句话吐槽2015年度十大烂片,2015-12-21,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
1271,2015最佳英美剧盘点，收藏寒假练听力丨附下载资源,2015-12-20,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
1272,完整收藏版丨1895年至今，全球电影票房总排行Top50【高清下载版】,2015-12-18,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
1273,喂！110吗？我妈朋友圈又发污鸡汤了！！,2015-12-17,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
1274,恩，是时候弃剧『芈月传』了！,2015-12-16,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...


In [109]:
# tagging 标记
"""
tagging_list = ["","马化腾", "腾讯", "微众银行","腾讯复星","腾讯风控","腾讯支付","WeChat","We Remit",\
                "公益日","红包",\
                "腾讯财付通","鹅厂","QQ钱包","QQ红包","QQ",\
                "只有一","大咖","听说","图片","照片","小编",\
                "洗钱", "黑产","被骗","腾讯安全课","诈骗", "炒股","神秘兼职","神秘组织",\
                "财付通", "品牌",\
                "收款",\
                "报告",\
                "银行卡","理财","选股","发票","基金",\
                "区块链","金融云","O2O","农产品","家乡",\
                "数据", "数据赋能", "智能", "数字孪生", "智慧大脑",\
                "出行","乘车","公交","乘车码", "智慧地铁",\
                "高峰论坛", "智库",\
                "央行","新规", \
                "微信", "微信支付", "跨境支付", "移动支付","非银行支付","电子支付",\
                "互联网金融", "金融科技","互联网＋","互联网+金融","普惠金融","虚拟银行",\
                "开放","生态","复杂","互联网思维","全球合作伙伴",\
                "联合国", "城市", "粤港澳大湾区", "平台", "可持续发展", "未来", "绿色",\
                "医护","防护服","小时","武汉","危机","新冠肺炎", "疫", "疫情", "复工","停课","宅经济",\
                "基建","新基建"] #overwritable
 """               

tagging_list = ["新媒体",\
                "传媒",\
                "数据",\
                "短视频",\
                "大数据",\
                "新媒体"]







v_v_list = []

for tag in tagging_list:
    index_list = df_前50页 [ df_前50页.标题.str.contains(tag) ].index.tolist()
    v_v_pairs = pd.DataFrame({tag:index_list}).melt().set_index("value")
    v_v_list.append(v_v_pairs)

df_cat = v_v_list[0]
for d in v_v_list:
    df_cat.update(d)
    
# 尚未标记内容
df_前50页.loc [ df_cat.query('variable==""').index ]

,标题,发布时间,链接
value,,,


In [126]:
df_1000条[df_1000条.duplicated()]

,标题,发布时间,链接
72,内容产品的快闪造节运动：被收割了注意力的我们，都是以消费填补焦虑,2018-11-25,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
73,《柯南》剧场版里的物联网恐怖袭击，能成真吗？,2018-11-25,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
74,如果有一天，你失去了自己的个人微信号,2018-11-21,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
75,CNN为什么能告赢特朗普,2018-11-21,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
76,为什么粉丝会热衷于为偶像“造”数据？,2018-11-20,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
77,为什么超级英雄选择从事新闻业？,2018-11-18,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
78,“90后开始失去了”——一个群体童年的消逝,2018-11-16,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...


In [85]:
df_url_out[~df_url_out.duplicated()]

,title,create_time,link
0,助你少走弯路！2020Fall美国传播与媒介研究类PhD申请分享,2020-04-10,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
1,“云监工”变“云守望” 守护和见证一座城市的复苏,2020-02-16,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
2,如果要用八个字总结2019年的互联网，你的答案是什么？,2020-01-07,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
3,“央视频”不一young：跨年“遥控器” 打造全国全民大联欢,2020-01-04,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
4,回顾与前瞻：新中国成立70年来的新闻传播史研究,2019-10-28,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
...,...,...,...
1270,一句话吐槽2015年度十大烂片,2015-12-21,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
1271,2015最佳英美剧盘点，收藏寒假练听力丨附下载资源,2015-12-20,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
1272,完整收藏版丨1895年至今，全球电影票房总排行Top50【高清下载版】,2015-12-18,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...
1273,喂！110吗？我妈朋友圈又发污鸡汤了！！,2015-12-17,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...


In [128]:
df_前1000章分类 = df_1000条.loc[0:1000].join(df_cat).replace("", np.nan).fillna("无法分类")
df_前1000章分类 

,标题,发布时间,链接,variable
0,助你少走弯路！2020Fall美国传播与媒介研究类PhD申请分享,2020-04-10,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,无法分类
1,“云监工”变“云守望” 守护和见证一座城市的复苏,2020-02-16,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,无法分类
2,如果要用八个字总结2019年的互联网，你的答案是什么？,2020-01-07,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,无法分类
3,“央视频”不一young：跨年“遥控器” 打造全国全民大联欢,2020-01-04,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,无法分类
4,回顾与前瞻：新中国成立70年来的新闻传播史研究,2019-10-28,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,无法分类
...,...,...,...,...
996,考研福利：2016年度热点事件学理性解读（下）,2016-12-23,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,无法分类
997,再见，我去考研了！,2016-12-23,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,无法分类
998,考研福利：2016年度热点事件学理性解读（上）,2016-12-22,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,无法分类
999,雾霾成了中国大众文化的一部分,2016-12-22,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,无法分类


In [129]:
df_前1000章分类 [df_1000条.loc[0:1000].标题.str.contains("新媒体")]

,标题,发布时间,链接,variable
88,那些称霸新媒体行业的旧媒体人,2018-11-03,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,新媒体
147,下半年，新媒体人最全求职攻略,2018-09-05,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,新媒体
204,新媒体人：20岁的年纪，40岁的颈椎,2018-07-20,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,新媒体
225,新媒体人的一天，哈哈哈哈哈哈哈哈哈,2018-07-06,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,新媒体
292,【征集】第七届“新媒体与全球信息社会的公正传播”学术研究营,2018-05-02,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,新媒体
301,如何斩获心仪的新媒体offer，大厂工作法你需要了解一下,2018-04-18,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,新媒体
363,微博热搜消失的第三天，新媒体人去哪儿追热点？,2018-01-30,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,新媒体
372,2017新媒体圈里那些轰轰烈烈的大事件,2018-01-18,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,新媒体
377,严峻的2018，新媒体有这四个系统性风险,2018-01-14,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,新媒体
415,2017-2018新媒体发展60大观察,2017-12-14,http://mp.weixin.qq.com/s?__biz=MzI2MzExMTQ3OA...,新媒体


In [130]:
df_无法分类和新媒体分类 = df_前1000章分类.groupby(by="variable").agg({"标题":"count"}).sort_values(by="标题", ascending=False)
df_无法分类和新媒体分类

,标题
variable,
无法分类,966
新媒体,35


In [133]:
df_无法分类和新媒体分类.to_excel("前1000章_传媒志_无法分类和新媒体分类_结果.xlsx",\
                sheet_name="传媒志新媒体分类结果")

In [134]:
df_前1000章分类 [df_1000条.loc[0:1000].标题.str.contains("新媒体")].to_excel("前1000章_传媒志_新媒体分类_结果.xlsx",\
                sheet_name="传媒志新媒体分类结果")

In [136]:
    with pd.ExcelWriter(r'C:\Users\win10\Desktop\data\前1000章整合.xls') as writer:
        df_1000条.loc[0:1000].to_excel(writer, sheet_name='传媒志结果')        
        df_无法分类和新媒体分类.to_excel(writer, sheet_name='传媒志新媒体与无法分类结果')
        df_前1000章分类[df_1000条.loc[0:1000].标题.str.contains("新媒体")].to_excel(writer, sheet_name='传媒志新媒体分类结果')
    
  

## 输出

In [95]:
df_account.columns.name = "rel_accounts"
df_o.columns.name = "url_cat"
df_stats.columns.name = "stats"


In [96]:
_df_.columns.name

'rel_accounts'

In [46]:
# Get the xlsxwriter workbook and worksheet objects.  
with pd.ExcelWriter(fn["output"]["公众号_xlsx"].format(公众号=公众号)) as writer:
    workbook  = writer.book

    for _df_ in [df_account]:
        _df_.to_excel(writer, sheet_name = _df_.columns.name)
